In [1]:
#%pip install protobuf==3.20.1
%pip install -q transformers sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 49.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 66.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 49.5 MB/s eta 0:00:00


In [1]:
QPATH = "Quantlet/code_description2project_description"

In [2]:
import sys
IN_COLAB = 'google.colab' in sys.modules

import os
if IN_COLAB:
  os.chdir(f'/content/drive/MyDrive/ColabNotebooks/IRTG/Encode_the_Qode/Encode-the-Qode/{QPATH}')

sys.path.append('../src')

In [3]:
!ls ../src

preprocessing_utils.py


In [4]:
import pickle
import json
import re
import sys
from IPython.display import display

from tqdm import tqdm
import pandas as pd
import numpy as np


import matplotlib.pyplot as plt
import seaborn as sns

import torch
from torch.utils.data import  DataLoader
from transformers import AutoTokenizer, AutoModelWithLMHead, SummarizationPipeline
from transformers import AdamW

import importlib
import preprocessing_utils
importlib.reload(preprocessing_utils)

<module 'preprocessing_utils' from '/content/drive/MyDrive/ColabNotebooks/IRTG/Encode_the_Qode/Encode-the-Qode/Quantlet/code_description2project_description/../src/preprocessing_utils.py'>

In [5]:
parsed_Qs_file = '../../data/preprocessed/Quantlet/Parsed_Qs_19032023.pkl'
dataset = preprocessing_utils.QuantletDataset(parsed_Qs_file)

CLEAN_UP = True
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
model=AutoModelWithLMHead.from_pretrained("SEBIS/code_trans_t5_large_source_code_summarization_python_multitask")
tokenizer=AutoTokenizer.from_pretrained("SEBIS/code_trans_t5_large_source_code_summarization_python_multitask",
                                        skip_special_tokens=False)
model.to(device)
print(device)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1362: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


In [9]:
from transformers import AutoTokenizer, T5Model

tokenizer = AutoTokenizer.from_pretrained("t5-small")
model = T5Model.from_pretrained("t5-small")

input_ids = tokenizer(
    "Studies have been shown that owning a dog is good for you", return_tensors="pt"
).input_ids  # Batch size 1
decoder_input_ids = tokenizer("Studies show that", return_tensors="pt").input_ids  # Batch size 1

# preprocess: Prepend decoder_input_ids with start token which is pad token for T5Model.
# This is not needed for torch's T5ForConditionalGeneration as it does this internally using labels arg.
decoder_input_ids = model._shift_right(decoder_input_ids)

# forward pass
outputs = model(input_ids=input_ids, decoder_input_ids=decoder_input_ids)
last_hidden_states = outputs.last_hidden_state

In [25]:
input_ids = tokenizer(
    "Studies have been shown that owning a dog is good for you", #return_tensors="pt"
)  # Batch size 1
decoder_input_ids = tokenizer("Studies show that", #return_tensors="pt"
                              )

In [28]:
data_loader = DataLoader(dataset, batch_size=1, shuffle=False)

In [26]:
input_ids

{'input_ids': [16888, 81, 128, 3658, 38, 666, 78, 15, 4908, 34, 680, 33, 197, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [24]:
decoder_input_ids

[16888, 504, 38, 1]

In [27]:
tokenizer.tokenize("Studies have been shown that owning a dog is good for you")

['▁Studies',
 '▁have',
 '▁been',
 '▁shown',
 '▁that',
 '▁own',
 'ing',
 '▁a',
 '▁dog',
 '▁is',
 '▁good',
 '▁for',
 '▁you']